In [1]:
import cv2
import os   
import numpy as np
import pywt
import matplotlib.pyplot as plt
from skimage.filters import gabor

In [2]:
# Lista de nombres de archivo de imágenes en el directorio
nombres_imagenes_normales = os.listdir('./data')

In [11]:
# Función para aplicar la transformada de Fourier a una imagen
def aplicar_fft_a_imagen(imagen, filtro_gaussiano=True, sigma=1.0):
    # Aplicar la transformada de Fourier 2D
    imagen_fft = np.fft.fft2(imagen)

    # Desplazar el componente de baja frecuencia al centro
    imagen_fft_desplazada = np.fft.fftshift(imagen_fft)

    # Calcular el logaritmo de la magnitud para una mejor visualización
    magnitud_fft = np.log(np.abs(imagen_fft_desplazada) + 1)

    # Aplicar un filtro Gaussiano para eliminar ruido
    if filtro_gaussiano:
        # Crear un filtro Gaussiano en el dominio de la frecuencia
        rows, cols = imagen.shape
        x = np.linspace(-cols/2, cols/2, cols)
        y = np.linspace(-rows/2, rows/2, rows)
        x, y = np.meshgrid(x, y)
        filtro_gaussiano = np.exp(-(x**2 + y**2) / (2 * sigma**2))

        # Aplicar el filtro Gaussiano a la imagen en el dominio de la frecuencia
        imagen_fft_filtrada = imagen_fft_desplazada * filtro_gaussiano

        # Deshacer el desplazamiento
        imagen_fft_filtrada = np.fft.ifftshift(imagen_fft_filtrada)

        # Aplicar la transformada inversa de Fourier
        imagen_filtrada = np.fft.ifft2(imagen_fft_filtrada).real

        return magnitud_fft, imagen_filtrada
    else:
        return magnitud_fft

def aplicar_wav_a_imagen(imagen, wavelet='haar', level=1, sigma=0.1):
    # Convertir la imagen a escala de grises si es una imagen a color
    if len(imagen.shape) == 3:
        imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

    # Normalizar los valores de píxeles al rango [0, 1]
    imagen = imagen / 255.0

    # Aplicar 2D Discrete Wavelet Transform
    coeffs = pywt.wavedec2(imagen, wavelet, level=level)

    # Reconstruct the image from the coefficients
    imagen_wavelet = pywt.waverec2(coeffs, wavelet)

    rows, cols = imagen_wavelet.shape
    x = np.linspace(-cols/2, cols/2, cols)
    y = np.linspace(-rows/2, rows/2, rows)
    x, y = np.meshgrid(x, y)
    filtro_gaussiano = np.exp(-(x**2 + y**2) / (2 * sigma**2))

    # Aplicar el filtro Gaussiano a la imagen
    imagen_wavelet_filtrada = imagen_wavelet * filtro_gaussiano

    return imagen_wavelet, imagen_wavelet_filtrada

In [14]:
# Procesar cada imagen en el directorio
for nombre_imagen in nombres_imagenes_normales:
    ruta_imagen = os.path.join("./data", nombre_imagen)

    # Cargar la imagen utilizando OpenCV
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Convierte la imagen a escala de grises

    # Aplicar la transformada de Fourier a la imagen
    imagen_fft = aplicar_fft_a_imagen(imagen)

    # Guardar la imagen procesada en un nuevo archivo
    nombre_imagen_salida = f'fft_{nombre_imagen}'
    ruta_imagen_salida = os.path.join("./Preprocesado_fft/", nombre_imagen_salida)
    cv2.imwrite(ruta_imagen_salida, imagen_fft[1])

print("Procesamiento de imágenes completado.")

Procesamiento de imágenes completado.


In [14]:
# Con gabor
frecuencia = 0.4

for nombre_imagen in nombres_imagenes_normales:
    ruta_imagen = os.path.join("./data", nombre_imagen)

    # Cargar la imagen utilizando OpenCV
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Convierte la imagen a escala de grises
    # Aplicar la transformada de Gabor a la imagen
    imagen_wav, _ = gabor(imagen, frecuencia)

    # Guardar la imagen procesada en un nuevo archivo
    nombre_imagen_salida = f'gabor_{nombre_imagen}'
    ruta_imagen_salida = os.path.join("./Preprocesado_gabor/", nombre_imagen_salida)
    cv2.imwrite(ruta_imagen_salida, np.abs(imagen_wav))

In [15]:
total = 0

for nombre_imagen in nombres_imagenes_normales:
    ruta_imagen = os.path.join("./data", nombre_imagen)
    ruta_imagen2 = os.path.join("./Preprocesado_gabor", f'gabor_{nombre_imagen}')

    # Cargar la imagen utilizando OpenCV
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Convierte la imagen a escala de grises
    imagen2 = cv2.imread(ruta_imagen2, cv2.IMREAD_GRAYSCALE)

    # Aplicar la transformada de Gabor a la imagen
    total += np.abs(imagen - imagen2)

np.mean(np.sum(total/len(nombres_imagenes_normales)))

7951.5109523809515

In [10]:
(total/len(nombres_imagenes_normales))

array([[0.04619048, 0.01690476, 0.0152381 , ..., 0.04809524, 0.0602381 ,
        0.02261905],
       [0.0302381 , 0.03428571, 0.01595238, ..., 0.02738095, 0.04904762,
        0.045     ],
       [0.04880952, 0.03333333, 0.02619048, ..., 0.02928571, 0.00880952,
        0.00142857],
       ...,
       [0.01619048, 0.01119048, 0.00119048, ..., 0.0252381 , 0.03095238,
        0.04571429],
       [0.0502381 , 0.05880952, 0.01595238, ..., 0.00190476, 0.02357143,
        0.03880952],
       [0.03595238, 0.03690476, 0.04404762, ..., 0.05214286, 0.0602381 ,
        0.0047619 ]])